In [1]:
# AlvaroMLProject
## This is my blank project to synchronize with Github for the final Capstone Project for IBM Data Science
import pandas as pd
import numpy as np

print("Hello Capstone Project Course!")


Hi, this is my blank notebook for the 1 week
